In [1]:
import pandas as pd

# Bitcoin analysis
El dataset original tiene datos cada 15 minutos
Dado que el analisis del proyecto consiste en 48 hs despues, no hay razon para detalle
Se toma como el valor +48hs el horario de cierre del mercado financiero en usa (16hs hora local)
de manera totalmente arbitraria

In [2]:
btc_df = pd.read_csv('../data/btc_15m_data_2018_to_2025.csv')
btc_df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2018-01-01 00:00:00.000000,13715.65,13715.65,13400.01,13556.15,123.616013,2018-01-01 00:14:59.999000,1.675545e+06,1572,63.227133,8.576108e+05,0
1,2018-01-01 00:15:00.000000,13533.75,13550.87,13402.00,13521.12,98.136430,2018-01-01 00:29:59.999000,1.321757e+06,1461,47.686389,6.422812e+05,0
2,2018-01-01 00:30:00.000000,13500.00,13545.37,13450.00,13470.41,79.904037,2018-01-01 00:44:59.999000,1.078825e+06,1000,43.710406,5.900347e+05,0
3,2018-01-01 00:45:00.000000,13494.65,13690.87,13450.00,13529.01,141.699719,2018-01-01 00:59:59.999000,1.917783e+06,1195,73.897993,1.000614e+06,0
4,2018-01-01 01:00:00.000000,13528.99,13571.74,13402.28,13445.63,72.537533,2018-01-01 01:14:59.999000,9.778198e+05,898,34.257652,4.618369e+05,0


In [3]:
btc_df.shape

(268653, 12)

In [ ]:

print(btc_df.columns.tolist())
print("\nColumn name mapping (original -> snake_case):")
column_mapping = {col: col.lower().replace(' ', '_') for col in btc_df.columns}
btc_df.rename(columns=column_mapping, inplace=True)
btc_df.columns.tolist()

['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

Column name mapping (original -> snake_case):


['open_time',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'close_time',
 'quote_asset_volume',
 'number_of_trades',
 'taker_buy_base_asset_volume',
 'taker_buy_quote_asset_volume',
 'ignore']

In [5]:
# Convert close_time to datetime if it's not already

btc_df['close_time'] = pd.to_datetime(btc_df['close_time'])

# Extract only close and close_time columns
btc_df = btc_df[['close_time', 'close']].copy()

# Filter for entries at market closing time (15:59:59.999000 or 16:00:00)
# We'll look for times that are close to 16:00 (15:59:59.999000)
market_close_time = pd.Timestamp('15:59:59.999000').time()

# Filter for the closing time entries
btc_df = btc_df[btc_df['close_time'].dt.time == market_close_time].copy()

print(btc_df.head())

                 close_time     close
63  2018-01-01 15:59:59.999  13247.00
159 2018-01-02 15:59:59.999  13690.03
255 2018-01-03 15:59:59.999  15033.77
344 2018-01-04 15:59:59.999  14353.01
440 2018-01-05 15:59:59.999  16087.72


## Trump tweets analysis

In [6]:
trump_df = pd.read_csv('../data/trump_tweets.csv')
trump_df.head()

,id,text,device,favorites,retweets,date
0,98455000000000000,Republicans and Democrats have both created ou...,TweetDeck,49,255,8/2/2011 18:07
1,1234650000000000000,I was thrilled to be back in the Great city of...,Twitter for iPhone,73748,17404,3/3/2020 1:34
2,1218010000000000000,RT @CBS_Herridge: READ: Letter to surveillance...,Twitter for iPhone,0,7396,1/17/2020 3:22
3,1304880000000000000,The Unsolicited Mail In Ballot Scam is a major...,Twitter for iPhone,80527,23502,9/12/2020 20:10
4,1218160000000000000,RT @MZHemingway: Very friendly telling of even...,Twitter for iPhone,0,9081,1/17/2020 13:13


Vamos a convertir la fecha de ese formato al ISO Date para ser equivalente al del otro df

In [ ]:
    trump_df = trump_df[['date', 'text', 'favorites', 'retweets']]
    trump_df = trump_df 
    trump_df['date'] = pd.to_datetime(trump_df['date'], format='%m/%d/%Y %H:%M')
    trump_df['date'].dtype

dtype('<M8[ns]')

Y vamos a filtrar las que ocurren solo despues de la fecha del primer tweet

In [8]:
# Filter tweets after 2018-01-01 00:00:00
btc_start_date = btc_df['close_time'].min()
trump_df = trump_df[trump_df['date'] >= btc_start_date].copy()


In [9]:
# Filter out tweets that contain only URLs or are mostly URLs
import re

def is_mostly_url(text):
    """Check if text is mostly URLs (more than 80% of the text)"""
    if not text or pd.isna(text):
        return True
    
    # Find all URLs in the text
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_pattern, text)
    
    if not urls:
        return False
    
    # Calculate total length of URLs
    total_url_length = sum(len(url) for url in urls)
    
    # If URLs make up more than 80% of the text, consider it mostly URLs
    return total_url_length / len(text) > 0.8

# Filter out tweets that are mostly URLs
trump_df = trump_df[~trump_df['text'].apply(is_mostly_url)].copy()

print(f"Filtered dataset shape: {trump_df.shape}")
print(f"Sample of remaining tweets:")
print(trump_df['text'].head())


Filtered dataset shape: (22451, 4)
Sample of remaining tweets:
1    I was thrilled to be back in the Great city of...
2    RT @CBS_Herridge: READ: Letter to surveillance...
3    The Unsolicited Mail In Ballot Scam is a major...
4    RT @MZHemingway: Very friendly telling of even...
5    RT @WhiteHouse: President @realDonaldTrump ann...
Name: text, dtype: object


Y para BTC vamos a filtrar para quedarnos con las mediciones hasta la fecha del ultimo tweet

In [10]:
trump_last_date = trump_df['date'].max()
trump_first_date = trump_df['date'].min()
btc_df = btc_df[btc_df['close_time'] <= trump_last_date + pd.Timedelta(days=7)].copy()
btc_df = btc_df[btc_df['close_time'] >= trump_first_date - pd.Timedelta(days=7)].copy()

In [ ]:
from google.cloud import language_v2


LANGUAGE_CODE = "en"

API_KEY = ""

client = language_v2.LanguageServiceClient(
    client_options={"api_key": API_KEY}
)


def moderate_text(text_content):

    document = {
        "content": text_content,
        "type_": "PLAIN_TEXT",
        "language_code": LANGUAGE_CODE,
    }
    response = client.moderate_text(
        request={"document": document}
    )

    # Create dictionary with category name and confidence pairs using comprehension
    return {cat.name: cat.confidence for cat in response.moderation_categories}
    


In [12]:
# Moderate text for all tweets and add categories as columns
print("Moderating text for all tweets...")

# Get all unique categories first by moderating a sample tweet
sample_categories = moderate_text(trump_df['text'].iloc[0])
category_names = list(sample_categories.keys())

# Initialize columns for each category
for category in category_names:
    trump_df[category] = 0.0

# Trim to 10 items for testing to avoid high API costs
trump_df = trump_df.head(10).copy()
print(f"Trimmed dataset to {len(trump_df)} tweets for testing", trump_df)


# Moderate each tweet and populate the category columns
for idx, text in trump_df['text'].items():
    try:
        categories = moderate_text(text)
        for category, confidence in categories.items():
            trump_df.at[idx, category] = confidence
    except Exception as e:
        print(f"Error processing tweet at index {idx}: {e}")
        continue

print(f"Completed text moderation for {len(trump_df)} tweets")
trump_df.head()

Moderating text for all tweets...
Trimmed dataset to 10 tweets for testing                   date                                               text  \
1  2020-03-03 01:34:00  I was thrilled to be back in the Great city of...   
2  2020-01-17 03:22:00  RT @CBS_Herridge: READ: Letter to surveillance...   
3  2020-09-12 20:10:00  The Unsolicited Mail In Ballot Scam is a major...   
4  2020-01-17 13:13:00  RT @MZHemingway: Very friendly telling of even...   
5  2020-01-17 00:11:00  RT @WhiteHouse: President @realDonaldTrump ann...   
6  2020-02-01 16:14:00  Getting a little exercise this morning! https:...   
11 2020-10-23 04:30:00             RT @EricTrump: https://t.co/NcrNdOSfIV   
12 2020-03-08 04:01:00  RT @GOPChairwoman: The economic boom continues...   
13 2020-02-07 17:35:00  RT @TomFitton: Vindman's behavior is a scandal...   
14 2020-01-09 12:24:00           Thank you Elise! https://t.co/Y4Hb0zf5jk   

    favorites  retweets  Toxic  Insult  Profanity  Derogatory  Sexual  \
1   

,date,text,favorites,retweets,Toxic,Insult,Profanity,Derogatory,Sexual,"Death, Harm & Tragedy",Violent,Firearms & Weapons,Public Safety,Health,Religion & Belief,Illicit Drugs,War & Conflict,Politics,Finance,Legal
1,2020-03-03 01:34:00,I was thrilled to be back in the Great city of...,73748,17404,0.014622,0.010394,0.002904,0.003574,0.002435,0.059701,0.016304,0.018182,0.051813,0.023965,0.053885,0.036585,0.472222,0.867769,0.079710,0.435185
2,2020-01-17 03:22:00,RT @CBS_Herridge: READ: Letter to surveillance...,0,7396,0.050346,0.050961,0.023985,0.008934,0.010127,0.085399,0.773973,0.105263,0.976744,0.103131,0.019215,0.294118,0.350000,0.680934,0.081967,0.948357
3,2020-09-12 20:10:00,The Unsolicited Mail In Ballot Scam is a major...,80527,23502,0.258527,0.112138,0.058611,0.043741,0.015008,0.082609,0.237805,0.105263,0.594771,0.103131,0.101103,0.700000,0.058824,0.937500,0.515901,0.858108
4,2020-01-17 13:13:00,RT @MZHemingway: Very friendly telling of even...,0,9081,0.018771,0.012035,0.004276,0.003942,0.002470,0.207207,0.195122,0.018182,0.594771,0.034950,0.053885,0.057377,0.375000,0.912500,0.079710,0.333333
5,2020-01-17 00:11:00,RT @WhiteHouse: President @realDonaldTrump ann...,0,25048,0.016713,0.011194,0.004276,0.005187,0.002435,0.073482,0.186747,0.076923,0.090909,0.096305,0.926540,0.057377,0.141892,0.969512,0.102302,0.948357


In [13]:
trump_df.head(1)

,date,text,favorites,retweets,Toxic,Insult,Profanity,Derogatory,Sexual,"Death, Harm & Tragedy",Violent,Firearms & Weapons,Public Safety,Health,Religion & Belief,Illicit Drugs,War & Conflict,Politics,Finance,Legal
1,2020-03-03 01:34:00,I was thrilled to be back in the Great city of...,73748,17404,0.014622,0.010394,0.002904,0.003574,0.002435,0.059701,0.016304,0.018182,0.051813,0.023965,0.053885,0.036585,0.472222,0.867769,0.07971,0.435185


In [14]:
btc_df.head(1)

,close_time,close
63,2018-01-01 15:59:59.999,13247.0


In [ ]:
# Add BTC price columns for 24h and 48h after each tweet
print("Adding BTC price data for 24h and 48h after each tweet...")

# Initialize the new columns
trump_df['btc_24h_after'] = None
trump_df['btc_48h_after'] = None

# Process each tweet
for idx, row in trump_df.iterrows():
    tweet_date = row['date']
    # Calculate target dates (1 and 2 days after the tweet)
    tweet_day = tweet_date.date()
    target_24h_day = tweet_day + pd.Timedelta(days=1)
    target_48h_day = tweet_day + pd.Timedelta(days=2)
    
    
    # Find BTC entry for 1 day after (any hour on that day)
    btc_24h_mask = btc_df['close_time'].dt.date == target_24h_day
    print(btc_24h_mask, tweet_day, target_24h_day, target_48h_day)
    if btc_24h_mask.any():
        closest_24h_idx = btc_df[btc_24h_mask].index[0]
        trump_df.at[idx, 'btc_24h_after'] = btc_df.at[closest_24h_idx, 'close']
    
    
    btc_48h_mask = btc_df['close_time'].dt.date == target_48h_day
    if btc_48h_mask.any():
        closest_48h_idx = btc_df[btc_48h_mask].index[0]
        trump_df.at[idx, 'btc_48h_after'] = btc_df.at[closest_48h_idx, 'close']
    

print(f"Added BTC price data for {len(trump_df)} tweets")


Adding BTC price data for 24h and 48h after each tweet...
63        False
159       False
255       False
344       False
440       False
          ...  
105683    False
105779    False
105875    False
105971    False
106067    False
Name: close_time, Length: 1107, dtype: bool 2020-03-03 2020-03-04 2020-03-05
63        False
159       False
255       False
344       False
440       False
          ...  
105683    False
105779    False
105875    False
105971    False
106067    False
Name: close_time, Length: 1107, dtype: bool 2020-01-17 2020-01-18 2020-01-19
63        False
159       False
255       False
344       False
440       False
          ...  
105683    False
105779    False
105875    False
105971    False
106067    False
Name: close_time, Length: 1107, dtype: bool 2020-09-12 2020-09-13 2020-09-14
63        False
159       False
255       False
344       False
440       False
          ...  
105683    False
105779    False
105875    False
105971    False
106067    False
Name: c

In [20]:
# Find entries where BTC columns are not None
btc_data_available = trump_df[(trump_df['btc_24h_after'].notna()) | (trump_df['btc_48h_after'].notna())]
print(f"Found {len(btc_data_available)} tweets with BTC price data available")
btc_data_available.head()

Found 10 tweets with BTC price data available


,date,text,favorites,retweets,Toxic,Insult,Profanity,Derogatory,Sexual,"Death, Harm & Tragedy",...,Public Safety,Health,Religion & Belief,Illicit Drugs,War & Conflict,Politics,Finance,Legal,btc_24h_after,btc_48h_after
1,2020-03-03 01:34:00,I was thrilled to be back in the Great city of...,73748,17404,0.014622,0.010394,0.002904,0.003574,0.002435,0.059701,...,0.051813,0.023965,0.053885,0.036585,0.472222,0.867769,0.079710,0.435185,8700.0,9085.48
2,2020-01-17 03:22:00,RT @CBS_Herridge: READ: Letter to surveillance...,0,7396,0.050346,0.050961,0.023985,0.008934,0.010127,0.085399,...,0.976744,0.103131,0.019215,0.294118,0.350000,0.680934,0.081967,0.948357,8903.26,8631.95
3,2020-09-12 20:10:00,The Unsolicited Mail In Ballot Scam is a major...,80527,23502,0.258527,0.112138,0.058611,0.043741,0.015008,0.082609,...,0.594771,0.103131,0.101103,0.700000,0.058824,0.937500,0.515901,0.858108,10280.14,10698.03
4,2020-01-17 13:13:00,RT @MZHemingway: Very friendly telling of even...,0,9081,0.018771,0.012035,0.004276,0.003942,0.002470,0.207207,...,0.594771,0.034950,0.053885,0.057377,0.375000,0.912500,0.079710,0.333333,8903.26,8631.95
5,2020-01-17 00:11:00,RT @WhiteHouse: President @realDonaldTrump ann...,0,25048,0.016713,0.011194,0.004276,0.005187,0.002435,0.073482,...,0.090909,0.096305,0.926540,0.057377,0.141892,0.969512,0.102302,0.948357,8903.26,8631.95
